In [13]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn import datasets, metrics, model_selection, svm
import statsmodels.api as sm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<h4>Read in building and electical consumption data</H4>

In [14]:
# read in electricity consumption and building data
kwh_year_df = pd.read_csv('clean-data/kwh_year.csv')

In [15]:
kwh_year_df = kwh_year_df.drop(columns='Unnamed: 0')

In [181]:
kwh_year_df = kwh_year_df.drop(columns=['Service Address', 'Date'])

In [16]:
# read in electricity consumption data
electricity_df = pd.read_csv('data/GRU_Customer_Electric_Consumption.csv')

<h4>Identify colinear features</h4>

In [202]:
kwh_year_df.corr()

,avg_kwh,Latitude,Longitude,prop_id,Imprv_ID,Effective_YrBlt,Actual_YrBlt,Bldg_Num,Heated_SquareFeet,StatedArea,LATITUDE,LONGITUDE,POST_CODE,SHAPE_Length,SHAPE_Area,kwh_per_sqft,code_release
avg_kwh,1.000000,0.040056,-0.127483,-0.094862,-0.096427,0.009237,0.050600,-0.088637,0.030000,0.071025,0.040056,-0.080272,-0.061957,0.051801,0.070886,0.756406,0.012844
Latitude,0.040056,1.000000,0.012955,-0.252798,-0.269676,-0.147971,-0.173119,-0.249917,0.084560,0.098052,1.000000,0.012580,0.050522,-0.068329,0.098006,-0.077943,-0.145616
Longitude,-0.127483,0.012955,1.000000,-0.088834,-0.083928,0.007443,0.016086,-0.103413,-0.076855,0.019429,0.012955,-0.072087,0.099465,0.061216,0.019362,-0.128218,0.020579
prop_id,-0.094862,-0.252798,-0.088834,1.000000,0.999163,-0.328155,-0.485720,0.979195,-0.073447,-0.143555,-0.252798,0.831592,-0.077742,-0.102772,-0.143622,0.172702,-0.347216
Imprv_ID,-0.096427,-0.269676,-0.083928,0.999163,1.000000,-0.330094,-0.489333,0.976026,-0.075217,-0.143269,-0.269676,0.824767,-0.065282,-0.103277,-0.143335,0.176002,-0.349110
Effective_YrBlt,0.009237,-0.147971,0.007443,-0.328155,-0.330094,1.000000,0.848367,-0.295380,-0.198313,0.097112,-0.147971,-0.355596,-0.098143,0.101583,0.097163,-0.123043,0.998260
Actual_YrBlt,0.050600,-0.173119,0.016086,-0.485720,-0.489333,0.848367,1.000000,-0.448812,-0.031026,0.178475,-0.173119,-0.488636,-0.131531,0.188564,0.178468,-0.134784,0.856471
Bldg_Num,-0.088637,-0.249917,-0.103413,0.979195,0.976026,-0.295380,-0.448812,1.000000,-0.062558,-0.140084,-0.249917,0.756615,-0.040090,-0.102596,-0.140148,0.157644,-0.312866
Heated_SquareFeet,0.030000,0.084560,-0.076855,-0.073447,-0.075217,-0.198313,-0.031026,-0.062558,1.000000,0.162652,0.084560,-0.073115,0.009133,0.039347,0.162402,-0.238142,-0.187165
StatedArea,0.071025,0.098052,0.019429,-0.143555,-0.143269,0.097112,0.178475,-0.140084,0.162652,1.000000,0.098052,-0.132986,-0.026166,0.857624,0.999999,0.005519,0.106285


<h4>Remove colinear features</h4>

In [17]:
col_kwh_year_df = kwh_year_df.drop(columns=['Bldg_Num', 'StatedArea', 'SHAPE_Length', 'Effective_YrBlt', 'Actual_YrBlt', 'Imprv_ID', 'LATITUDE', 'LONGITUDE', 'POST_CODE']) 
col_kwh_year_df.columns

Index(['Service Address', 'avg_kwh', 'Date', 'Latitude', 'Longitude',
       'prop_id', 'Heated_SquareFeet', 'SHAPE_Area', 'kwh_per_sqft',
       'code_release'],
      dtype='object')

<h4>PCA</h4>

In [18]:
# standardizing data
features = ['avg_kwh', 'Latitude', 'Longitude', 'prop_id', 'Heated_SquareFeet',
       'SHAPE_Area', 'kwh_per_sqft', 'code_release']
# Separating out the features
x = col_kwh_year_df.loc[:, features].values
# Separating out the target
y = col_kwh_year_df.loc[:,['code_release']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [44]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2', 'pc3'])

In [45]:
finalDf = pd.concat([principalDf, col_kwh_year_df[['code_release']]], axis = 1)

In [46]:
year_list = col_kwh_year_df['code_release'].values

In [47]:
finalDf.head()

,pc1,pc2,pc3,code_release
0,-1.444113,-1.146119,1.210857,1973
1,-1.142488,-1.024339,1.233478,1973
2,1.631422,0.162212,1.339596,1973
3,-0.695765,-0.833394,1.250785,1973
4,-1.217239,-1.056461,1.230844,1973


<h4>Define Target variable and feature set</h4>

In [48]:
y = finalDf.pop('code_release')
X = finalDf

<h4>Train, test split and instantiate linear regression models</h4>

In [49]:
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=.25, random_state=1)

<h4>Set benchmark model</h4>

In [50]:
alpha = 3.0
linear = LinearRegression()
lasso = Lasso()
ridge = Ridge(alpha=alpha)
knn = KNeighborsRegressor(5)

In [51]:
folds = 15
#score = mean_squared_error()
linear_cv = cross_val_score(linear, X, y, cv=folds, scoring='neg_mean_squared_error')
lasso_cv = cross_val_score(lasso, X, y, cv=folds, scoring='neg_mean_squared_error')
ridge_cv = cross_val_score(ridge, X, y, cv=folds, scoring='neg_mean_squared_error')
knn_cv = cross_val_score(knn, X, y, cv=folds, scoring='neg_mean_squared_error')

In [52]:
lin_cv_mean = linear_cv.mean()
lasso_cv_mean = lasso_cv.mean()
ridge_cv_mean = ridge_cv.mean()
knn_cv_mean = knn_cv.mean()

print(lin_cv_mean, lasso_cv_mean, ridge_cv_mean, knn_cv_mean)

-60.29896947987664 -58.119610568264285 -59.90190851120759 -51.04358461538466


<h4>Ordinary Least Squares</h4>

In [53]:
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.mse_total)
# print(est2.summary())

165.7162859624917


<h4>Run random forest regressor</h4>

In [55]:
# instantiate RF and linear regression model
rf = RandomForestRegressor(n_estimators=100, max_depth=4, min_samples_split=5, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True)
rf.fit(X_train, y_train)
y_hat = rf.predict(X_test)

mse = mean_squared_error(y_test, y_hat)
mse

37.6139998445364

<h4>Generate Predictions for kwh consumption data</h4>

In [ ]:
'avg_kwh', 'Latitude', 'Longitude', 'prop_id', 'Heated_SquareFeet',
       'SHAPE_Area', 'kwh_per_sqft', 'code_release'

In [32]:
# remove non numerical columns 
electricity_df = electricity_df.drop(columns=['Service Address','Service City', 'Month', 'Year', 'Date', 'Location'])

In [37]:
electricity_df = electricity_df.dropna()

In [43]:
electricity_df.head()
# electricity_arr = electricity_df.to_numpy()

,KWH Consumption,Latitude,Longitude
0,1813.0,29.628666,-82.433346
1,1681.0,29.591773,-82.437221
2,8560.0,29.624492,-82.433521
3,1089.0,29.644607,-82.265500
4,1277.0,29.694047,-82.412384


<h4>Scale input Data</h4>

In [77]:
# standardizing data
features_input = ['KWH Consumption', 'Latitude', 'Longitude']
# Separating out the features
x_input = electricity_df.loc[:, features_input].values
# Standardizing the features
x_input = StandardScaler().fit_transform(x_input)

In [84]:
x_input[1]

array([ 0.00539479, -2.07607113, -1.50597764])

<h4>Generate predictions df</h4>

In [80]:
y_pred = rf.predict(x_input)

In [81]:
y_pred.shape

(9287688,)

In [82]:
y_pred_df = pd.DataFrame({'col_1': y_pred[:] })

In [87]:
# create column for predicted energy efficiency (higher number = more efficient)
preds_electricity_df = electricity_df
preds_electricity_df['preds'] = y_pred_df

In [89]:
preds_electricity_df.head()

,KWH Consumption,Latitude,Longitude,preds
0,1813.0,29.628666,-82.433346,2002.390361
1,1681.0,29.591773,-82.437221,2003.218251
2,8560.0,29.624492,-82.433521,2003.712840
3,1089.0,29.644607,-82.265500,1972.273439
4,1277.0,29.694047,-82.412384,1997.503866


<h4>Export predictions dataframe</h4>

In [91]:
# export predictions data for use in the carbon zero application
preds_electricity_df.to_csv(r'stationary-data/preds_electricity.csv', header=True)